## Comparison of the Glen-and-Isaacs and the Saltenis estimator

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import sobol_seq
from string import ascii_lowercase
import numba
import matplotlib.pyplot as plt

k = 6 # number of parameters inquired

# It is assumed the sm represents the dataframe on which we will be operating
a2 = [0, 0.5, 3, 9, 99, 99]
b3 = [6.52, 6.52, 6.52, 6.52, 6.52, 6.52]

### Defining the test functions

In [2]:
@numba.jit    
def A1(sm):
    dummyA1 = pd.DataFrame()
    for j in range(0,k):
        dummyA1[j] = np.prod(sm[sm.columns[0:j+1]], axis = 1)*(-1)**(j+1)
    return dummyA1.sum(axis=1)
        
def A2(sm):
    dummyA2 = pd.DataFrame()
    for j in range(0,k):
        dummyA2[j] = (abs(4*sm[sm.columns[j]]-2)+a2[j])/(1+a2[j])
    return (dummyA2.product(axis=1))
        
def B1(sm):
    dummyB1 = pd.DataFrame()
    for j in range(0,k):
        dummyB1[j] = (k-sm[sm.columns[j]])/(k-0.5)
    return dummyB1.product(axis=1)
        
def B2(sm):
    dummyB2 = pd.DataFrame()
    for j in range(0,k):
        dummyB2[j] = (sm[sm.columns[j]])**(1/k)
    return dummyB2.product(axis=1)*((1+1/k)**k)
        
def B3(sm):
    dummyB3 = pd.DataFrame()
    for j in range(0,k):
        dummyB3[j] = (abs(4*sm[sm.columns[j]]-2)+b3[j])/(1+b3[j])
    return (dummyB3.product(axis=1))
        
def C1(sm):
    dummyC1 = pd.DataFrame()
    for j in range(0,k):
        dummyC1[j] = pd.Series(abs(4*sm[sm.columns[j]]-2))
    return (dummyC1.product(axis=1))
        
def C2(sm):
    return (sm.product(axis=1)*2**k)

functions = [A1, A2, B1, B2, B3, C1, C2]

### And the analytical values

In [3]:
#AEA1

def Ek2(k):
    return((1/6)*(1-(1/3)**k)+(4/15)*((-1)**(k+1)*(1/2)**k+(1/3)**k))

def V(k):
    return((1/10)*(1/3)**k+(1/18)-(1/9)*(1/2)**(2*k)+(-1)**(k+1)*(2/45)*(1/2)**k)

for j in range(0, k):
    def Ej(j):
        return((1/6)*(1-(1/3)**(j))+(4/15)*((-1)**(j+1)*(1/2)**(j)+(1/3)**(j)))
              
    def T1(j):
        return((1/2)*((1/3)**(j-1)*(1-(1/3)**(k-j))))

    def T2(j):
        return((1/2)*((1/3)**j-(1/3)**k))
    
    def T3(j):
        return((3/5)*(4*(1/3)**(k+1)+(-1)**(j+k+1)*(1/2)**(k-j-2)*(1/3)**(j+1)))
    
    def T4(j):
        return((1/5)*((-1)**(j+2)*(1/3)*(1/2)**(j-2)-4*(1/3)**(j+1)))
    
    def T5(j):
        return((1/5)*((-1)**(k+1)*(1/3)*(1/2)**(k-2)+(-1)**(k+j)*(1/3)**(j+1)*(1/2)**(k-j-2)))
    
def A1ST(j):
    return((Ek2(k)-Ej(j)-(1/4)*(T1(j)-2*T2(j)+T3(j))-T4(j)-T5(j))/V(k))

#ÁEA2

def VjA2(j):
    return((1/3)/(1+a2[j])**2)

def VA2(j):
    productA2 = []
    for j in range(0, k):
        productA2.append(1+VjA2(j))
    return np.product(productA2)-1

def VnA2(j):
    return((VA2(j)+1)/(1+VjA2(j)))

def VTjA2(j):
    return VjA2(j)*(VnA2(j))

def A2ST(j):
    return(VTjA2(j)/VA2(j))

#AEB1

def p(j):
    return(12*(k-0.5)**2)
        
def B1ST(j):
    return((p(j)+1)**k/((p(j)+1)*((p(j)+1)**k-p(j)**k)))

#AEB2

def B2ST(j):
    return((k+1)**(2*k-2)/(((k+1)**(2*k)-(k**2+2*k)**k)))

#AEB3

def VjB3(j):
    return((1/3)/(1+b3[j])**2)

def VB3(j):
    productB3 = []
    for j in range(0, k):
        productB3.append(1+VjB3(j))
    return np.product(productB3)-1

def VnB3(j):
    return((VB3(j)+1)/(1+VjB3(j)))

def VTjB3(j):
    return VjB3(j)*(VnB3(j))

def B3ST(j):
    return(VTjB3(j)/VB3(j))

#AEC1

def C1ST(j):
    return 4**(k-1)/(4**k-3**k)

#AEC2
        
def C2ST(j):
    return 4**(k-1)/(4**k-3**k)

def create_dict(key, values):
    return dict(zip(key, values))

analyticalValues = [A1ST, A2ST, B1ST, B2ST, B3ST, C1ST, C2ST]

AE = []
AE_names = []
for w in analyticalValues:
    for j in range (0,k):
        AE.append(w(j))
        AE_names.append('AE' + str(w.__name__) + str(j+1))
AE_dic = create_dict(AE_names, AE)

### It is then time to define the sample and scrambled matrices

In [4]:
letters = []

for l in ascii_lowercase:
    letters.append(l)
    
p = 14
run = 50

n = 2

p_sample = []
p_sample_name = []

p_sample_Jan = []
p_sample_name_Jan = []

df = pd.DataFrame(sobol_seq.i4_sobol_generate(k, run*2**p-4))
df = df.sample(frac=1).reset_index(drop=True)

In [5]:
qamples = []
for s in range (2,p):
    qamples.append(df.iloc[run*(-4+2**s):run*(-4+2**(s+1))].reset_index(drop=True))
    
    GIMAE_mean = []
    CheckMAE_mean = []
    
    run_samples = []
    for r in range (run):
        run_samples.append(qamples[s-2].iloc[int(r*(len(qamples[s-2])/run)):int((r+1)*(len(qamples[s-2].\
        index)/run))].reset_index(drop=True))
        
        sample_Matrices = []
        for m in range (n):
            sample_Matrices.append(run_samples[r].iloc[int(m*(len(run_samples[r].index)/n)):int((m+1)*\
            (len(run_samples[r].index)/n))].reset_index(drop=True))
                
        sample_Matrices_dic = create_dict(letters, sample_Matrices)
        
            
        mixed_Matrices = []
        mm_names = []
        for sm in range (0,len(sample_Matrices)):
            for sm1 in range (0,len(sample_Matrices)):
                if sm == sm1:
                    continue
                else:
                    for c in sample_Matrices[sm]:
                        mixed_Matrices.append(sample_Matrices[sm].copy())
                        mixed_Matrices[len(mixed_Matrices)-1][c]=sample_Matrices[sm1].copy()[c]
                        mm_names.append(str(letters[sm] + letters[sm1] + str(c+1)))
        mixed_Matrices_dic = create_dict(mm_names, mixed_Matrices)
        
        matrices_dic = {**sample_Matrices_dic, **mixed_Matrices_dic}
        
        names = []
        names1 = []
        names2 = []
        values = []
        values1 = []
        values2 = []
        for f in functions:
            for ss, zs in sample_Matrices_dic.items():
                names.append(f.__name__+str(ss))
                values.append(f(zs))
    
            for sq, zq in mixed_Matrices_dic.items():
                names1.append(f.__name__+str(sq))
                values1.append(f(zq))
            
            for sM, zM in matrices_dic.items():
                names2.append(f.__name__+str(sM))
                values2.append(f(zM))
                
        f_SM_dic = create_dict(names, values)
        f_MM_dic = create_dict(names1, values1)
        f_matrices_dic = create_dict(names2, values2)
        
        g_SM_dic = dict(f_SM_dic)
        g_MM_dic = dict(f_MM_dic)
        for mk, mv in f_SM_dic.items():
            g_SM_dic[mk]=(mv-mv.mean())/(mv.var()**0.5)
        for fk1, fv1 in f_MM_dic.items():
            g_MM_dic[fk1]=(fv1-fv1.mean())/(fv1.var()**0.5)
        
        pk =[]
        pk_name = []
        cd = []
        cdM = []
        cd_name = []
        cdM_name = []
        for gk, gv in g_SM_dic.items():
            for gk2, gv2 in g_SM_dic.items():
                for j in range(1,k+1):
                    if gk2[0:2]==gk[0:2] and gk2[2]!=gk[2] and gk[2]=='a':
                        for gk1,gv1 in g_MM_dic.items():
                            for gk3,gv3 in g_MM_dic.items():
                                if gk3[0:2]==gk1[0:2]==gk[0:2] and gk3[2]!=gk1[2] and gk1[2]=='a' and gk1[-1]==gk3[-1]==str(j):
                                    pk.append(0.5*(gv*gv2+gv1*gv3).mean())
                                    cd.append(0.5*(gv*gv3+gv1*gv2).mean())
                                    cdM.append(0.5*(gv*gv1+gv2*gv3).mean())
                                    pk_name.append(str(gk3[0:2])+'pk'+str(j))
                                    cd_name.append(str(gk3[0:2])+'cd'+str(j))
                                    cdM_name.append(str(gk3[0:2])+'cd-'+str(j))
        pk_dic = create_dict(pk_name, pk)
        cd_dic = create_dict(cd_name, cd)
        cdM_dic = create_dict(cdM_name, cdM)
        
        ca=[]
        caM=[]
        ca_name=[]
        caM_name=[]
        for cdk, cdv in cd_dic.items():
            for cdMk, cdMv in cdM_dic.items():
                for pkk, pkv in pk_dic.items():
                    if pkk[0:2]==cdMk[0:2]==cdk[0:2] and pkk[-1]==cdMk[-1]==cdk[-1]:
                        ca.append((cdv-pkv*cdMv)/(1-pkv**2))
                        caM.append((cdMv-pkv*cdv)/(1-pkv**2))
                        ca_name.append(str(pkk[0:2])+'ca'+str(pkk[-1]))
                        caM_name.append(str(pkk[0:2])+'ca-'+str(pkk[-1]))
        ca_dic = create_dict(ca_name, ca)
        caM_dic = create_dict(caM_name, caM)
        
        ST = []
        ST_name = []
        for cdMk, cdMv in cdM_dic.items():
            for pkk, pkv in pk_dic.items():
                for cak, cav in ca_dic.items():
                    for caMk, caMv in caM_dic.items():
                        if caMk[0:2]==cak[0:2]==pkk[0:2]==cdMk[0:2] and caMk[-1]==cak[-1]==pkk[-1]==cdMk[-1]:
                            ST.append(1-cdMv+pkv*cav/(1-cav*caMv))
                            ST_name.append(str('GI_'+pkk[0:2]+'ST'+ pkk[-1]))
        ST_dic = create_dict(ST_name, ST)
        
        GIMAEs = []
        GIMAENames = []
        for ae, av in AE_dic.items():
            for Lk, Lv in ST_dic.items():
                if ae[-5:]==Lk[-5:]:
                    GIMAEs.append(abs(Lv-av))
                    GIMAENames.append('GIMAE'+ str(ae[2:4]) + str(ae[-1]))
        GIMAEs_dic = create_dict(GIMAENames, GIMAEs)
        
        GIMAE = []
        GIMAE_name = []
        for f in functions:
            validkeys2 = []
            for Lmk, Lmv in GIMAEs_dic.items():
                if Lmk[-3:-1]==f.__name__:
                    validkeys2.append(Lmk)
            z2 = dict(filter(lambda i2:i2[0] in validkeys2, GIMAEs_dic.items()))
            GIMAE.append(sum(z2.values())/len(z2))
            GIMAE_name.append('GIMAE'+f.__name__)
        GIMAE_dic = create_dict(GIMAE_name, GIMAE)
        GIMAE_mean.append(GIMAE_dic)
        
        Check=[]
        CheckName = []
        for f in functions:
            for j in range(1,k+1):
                difference = []
                for mk, mz in f_matrices_dic.items():
                    if mk[0:2]==f.__name__:
                        validkeys3 = []
                        for fk1 in f_MM_dic.keys():
                            if len(mk)==3: 
                                if mk[2] == 'a' and fk1[0:3]==mk[0:3] and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                            else: 
                                if fk1[0:3]==mk[0:3] and fk1[-1]==mk[-1] and fk1!=mk and fk1>=mk and fk1[-1]==str(j):
                                    validkeys3.append(fk1)
                        z1 = dict(filter(lambda i:i[0] in validkeys3, f_MM_dic.items()))
                        for zk, zv in z1.items():
                            difference.append(0.5*(((mz-zv)**2).mean())/mz.var())
                Check.append(sum(difference)/len(difference))
                CheckName.append('Jansen'+ str(f.__name__) +'ST'+str(j))
        Check_dic = create_dict(CheckName, Check)
                    
        CheckMAEs = []
        CheckMAENames = []
        for ae, av in AE_dic.items():
            for Jk, Jv in Check_dic.items():
                if ae[-5:]==Jk[-5:]:
                    CheckMAEs.append(abs(Jv-av))
                    CheckMAENames.append('CheckMAE'+ str(ae[2:4]) + str(ae[-1]))
        CheckMAEs_dic = create_dict(CheckMAENames, CheckMAEs)
    
        CheckMAE = []
        CheckMAE_name = []
        for f in functions:
            validkeys4 = []
            for Jmk, Jmv in CheckMAEs_dic.items():
                if Jmk[-3:-1]==f.__name__:
                    validkeys4.append(Jmk)
            z2 = dict(filter(lambda i4:i4[0] in validkeys4, CheckMAEs_dic.items()))
            CheckMAE.append(sum(z2.values())/len(z2))
            CheckMAE_name.append('CheckMAE'+f.__name__)
        CheckMAE_dic = create_dict(CheckMAE_name, CheckMAE)
        CheckMAE_mean.append(CheckMAE_dic)
        
    GIMAE_mean_dic = {Lmk1:[GIMAE_mean[Lmv1][Lmk1] for Lmv1 in range(len(GIMAE_mean))] for Lmk1 in GIMAE_mean[0].keys()}
    for Lmk1,Lmv1 in GIMAE_mean_dic.items():
        GIMAE_mean_dic[Lmk1] = sum(GIMAE_mean_dic[Lmk1])/len(GIMAE_mean_dic[Lmk1])
        
    CheckMAE_mean_dic = {Jmk1:[CheckMAE_mean[Jmv1][Jmk1] for Jmv1 in range(len(CheckMAE_mean))] for Jmk1 in CheckMAE_mean[0].keys()}
    for Jmk1,Jmv1 in CheckMAE_mean_dic.items():
        CheckMAE_mean_dic[Jmk1] = sum(CheckMAE_mean_dic[Jmk1])/len(CheckMAE_mean_dic[Jmk1])
        
    p_sample.append(GIMAE_mean_dic)
    p_sample_name.append((k+1)*2**s)
    p_sample_Jan.append(CheckMAE_mean_dic)
    p_sample_name_Jan.append((k+1)*2**(s-1))
p_sample_dic = create_dict(p_sample_name, p_sample)
p_sample_Jan_dic = create_dict(p_sample_name_Jan, p_sample_Jan)
GlenIsaacs = pd.DataFrame.from_dict(p_sample_dic)
Jansen_asym = pd.DataFrame.from_dict(p_sample_Jan_dic)

The Sobol matrix gets sliced in a way that every sample is twice bigger than the previous qamples list elements. The qamples matrix gets in turn sliced in 50 parts of equivalent size. The rationale behind this operation is generating an adequate numbers of runs whose output can be averaged to compensate for fluctuations.

Eventually, the two sample matrices are generated by slicing down the larger matrix in two parts of equal length. The code line sample_Matrices_dic creates a dictionary by appending the name in the alphabet sequence to the matrices in the order they have been generated (a, b). This operation can be ideally replicated in the case one wishes to compute more sample matrices for the estimators algorithm (ref. to the literature).

The mixed matrices (i.e. ab1, ab6, ba2, etc.) are generated by scrambling the columns of the sample matrices. The first if excludes counting the same matrix twice, the second lower level for loop replicates the matrices the number of parameters one has and for each scrambles the relative column. Finally, the name is appended dependent the original matrix, the matrix whose column has been used for the scrambling and the number of the column. The label is eventually associated to the mixed matrices in the same way as done for the sample matrices. The sample and mixed matrices are zipped together into a matrices_dic.

The functions are applied to the sets of matrices and the result of the operations are stored into dictionaries again. Now each item in the dictionaries f_MM_dic and f_matrices_dic is a vector rather than a matrix (f(a), f(b), f(ab4), etc.)

For each function, the sample matrices are selected ('if len(mk)==3:'). From the ensamble of the scramble matrices, those having the same function and starting letter are selected for the total sensivity index accounting. While a different starting letter is the condition for the first-order estimator (fk1[2]!=mk[2]). The selection is based on the names (keys). The dictionary is filtered according to these criteria and the Jansen estimator-to-variance ratio is computed for the higher order (Check and CheckR) and the Sobol estimator-to-variance ratio for the first order matrices (Check3 and Check3R).

Finally, the Mean Absolute Errors, the difference between the analytical value and the estimator figure, are computed and the values appended in a dictionary.

These figures are then averaged on the different parameters (from six figures to one) as well as runs and stored in a dictionary CheckMAE_dic and CheckMAEF_dic.

Which is in turn appended onto the final dictionary wrapping up all the experiments performed (up to the power of 2 'p' intially defined). This latter is eventually converted into a convenient pandas dataframe from which the trends can be easily plotted or statistically inference on the figures can be produced. The same applies to the Glen-and-Isaacs estimator.

In [11]:
for ind, row in GlenIsaacs.iterrows():
    for indR, row in Jansen_asym.iterrows():
        if ind[2:] == indR[-5:]:
            x_vals = GlenIsaacs.columns.values
            x2_vals = Jansen_asym.columns.values
            y1 = GlenIsaacs.loc[ind]
            y2 = Jansen_asym.loc[indR]
            for i6 in range(0, len(x_vals), 1):
                plt.loglog(x_vals[i6:i6+2], y1[i6:i6+2], c = "b", marker = "o", label = 'GI' if i6 == 0 else '')
                plt.loglog(x2_vals[i6:i6+2], y2[i6:i6+2], c = "r", marker = "o", label = 'Jan' if i6 == 0 else '')
                plt.title('Glen&IsaacsVsJansen_asym' + '_ST_' +str(ind[-2:]))
                plt.xlim(110,58000)
                plt.ylim(0,1)
            plt.legend()
            plt.show()

The plot finally allows to explore the trend across the sample dimension inquired.